In [263]:
import numpy as np
from sklearn.preprocessing import MinMaxScaler

In [264]:
def generate(filename):
    file = open(filename)
    lines = file.readlines()
    
    X = []
    y = []
    
    for line in lines:
        var = line.split()
        X.append([float(x) for x in var[:-1]])
        y.append(int(var[-1]))
        
    unique = len(np.unique(y))
    
    Y = []
    for target in y:
        instance = np.zeros(unique)
        instance[target-1] = 1
        Y.append(instance)
        

    X = np.array(X)
    Y = np.array(Y)
    
    file.close()
    
    return X,np.array(Y)

In [265]:
X_train,Y_train = generate('trainNN.txt')
X_test, Y_test  =  generate('testNN.txt')

In [266]:
np.random.seed(42)
class L_Layer_NN:
    def __init__(self,L,k,learning_rate = 0.03):
        self.L = L
        self.k = k
        self.W = []
        self.W.append([])
        self.lr = 0.03
        self.k = np.insert(self.k,0,values=0,axis=0)
        print(self.k)
    
    def sigmoid(self,z):
        return 1/(1 + np.exp(-z))

    def derivative(self,z):
        return self.sigmoid(z)*(1-self.sigmoid(z))
    
    def fit(self,X_train,Y_train,max_epoch=1000):
        N = len(X_train)
        
        self.k[0] = X_train.shape[1]
        
        for r in range(1,self.L + 1):
            self.W.append([])
            self.W[r].append([])
            for j in range(1,self.k[r]+1):
                self.W[r].append([])
                for k in range(self.k[r - 1] + 1):
                    self.W[r][j].append(np.random.uniform(0, 1))
                    
        X_train = np.insert(X_train,0,values=1,axis=1)
        Y_train = np.insert(Y_train,0,values=0,axis=1)
        
        for epoch in range(max_epoch):
            y_m = []
            
            for i in range(N):
                x_i = X_train[i]
                
                y = []
                y.append(x_i)
                
                v = []
                v.append([])
                
                for r in range(1,self.L+1):
                    v_r = []
                    v_r.append([])
                    y_r = []
                    y_r.append(1)
                    for j in range(1,self.k[r]+1):
                        dot_product = np.dot(np.array(self.W[r][j]),np.array(y[r-1]))
                        v_r.append(dot_product)
                        y_r.append(self.sigmoid(dot_product))
                        
                    v.append(v_r)
                    y.append(y_r)
                    
                
                delta_L = []
                delta_L.append([])
                
                y_m.append(y[self.L])
                
                for j in range(1, self.k[self.L]+1):
                    e_j = y[self.L][j] - Y_train[i][j]
                    delta_L.append(e_j*self.derivative(v[self.L][j]))
                    
                
                delta = []
                delta.append([])
                
                for r in range(1,self.L+1):
                    if r == self.L :
                        delta.append(delta_L)
                    else:
                        temp = []
                        temp.append([])
                        for k in range(self.k[r]):
                            temp.append(0)
                        delta.append(temp)
                        
                
                for r in range(self.L,1,-1):
                    for j in range(1,self.k[r-1]+1):
                        e_j = 0
                        for k in range(1,self.k[r]+1):
                            e_j += delta[r][k]*self.W[r][k][j]
                        
                        delta[r-1][j] = e_j*self.derivative(v[r-1][j])
                
                for r in range(1,self.L+1):
                    for j in range(1,self.k[r]+1):
                        del_w = np.multiply(y[r-1],delta[r][j])
                        del_w = np.multiply(-self.lr,del_w)
                        self.W[r][j] = self.W[r][j] + del_w
            
            
            J = 0
            for i in range(0, N):
                e_i = 0.5*np.sum((y_m[i][1:]-Y_train[i][1:])**2,axis=0)
                J += e_i
            print("Epoch " + str(epoch) + ", Cost " + str(J))
            if J < 10:
                break
            
        pass
    
    
    def predict(self,X,Y):
        X = np.insert(X,0,values=1,axis=1)
        Y = np.insert(Y,0,values=0,axis=1)
        
        N = len(X)
        true = 0
        
        for i in range(N):
            x_i = X[i]
                
            y = []
            y.append(x_i)
                
            for r in range(1,self.L+1):
                y_r = []
                y_r.append(1)
                for j in range(1,self.k[r]+1):
                    dot_product = np.dot(np.array(self.W[r][j]),np.array(y[r-1]))
                    y_r.append(self.sigmoid(dot_product))
                        
                y.append(y_r)
            
            
            if np.argmax(y[self.L][1:]) == np.argmax(Y[i][1:]):
                true += 1
                
        return true/N

In [268]:
output_neurons = Y_train.shape[1] 

model = L_Layer_NN(2,[4,output_neurons])

min_max_scaler = MinMaxScaler()  # min max scaler
X_train = min_max_scaler.fit_transform(X_train)
X_test = min_max_scaler.fit_transform(X_test)

model.fit(X_train,Y_train)

[0 4 4]
Epoch 0, Cost 301.15325014727136
Epoch 1, Cost 187.78291160056423
Epoch 2, Cost 185.72145264032113
Epoch 3, Cost 184.20586948364297
Epoch 4, Cost 182.42638809586734
Epoch 5, Cost 180.24817636810383
Epoch 6, Cost 177.51982952777465
Epoch 7, Cost 174.06648831178538
Epoch 8, Cost 169.7418077570964
Epoch 9, Cost 164.5482947014756
Epoch 10, Cost 158.74113181742248
Epoch 11, Cost 152.75475379414027
Epoch 12, Cost 147.00780401908617
Epoch 13, Cost 141.7817112828238
Epoch 14, Cost 137.20314786527607
Epoch 15, Cost 133.27652111190204
Epoch 16, Cost 129.93370323064045
Epoch 17, Cost 127.07882693148764
Epoch 18, Cost 124.6163284141515
Epoch 19, Cost 122.46324304429453
Epoch 20, Cost 120.55202277785419
Epoch 21, Cost 118.82918438075716
Epoch 22, Cost 117.2527296208307
Epoch 23, Cost 115.78959641084482
Epoch 24, Cost 114.41354547207148
Epoch 25, Cost 113.1035316590511
Epoch 26, Cost 111.84248937041136
Epoch 27, Cost 110.61643762870447
Epoch 28, Cost 109.41382003368996
Epoch 29, Cost 108.225

In [269]:
model.predict(X_train,Y_train)

1.0

In [270]:
model.predict(X_test,Y_test)

1.0